In [0]:
import pandas as pd
import numpy as np

In [2]:
apps = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projetos Autonomos/Machine Learn/Recomendador de app na APPLE STORE/AppleStore.csv')
apps_descricao = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Projetos Autonomos/Machine Learn/Recomendador de app na APPLE STORE/appleStore_description.csv')
apps.head()

,Unnamed: 0,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,1,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,38,5,10,1
1,2,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,37,5,23,1
2,3,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,37,5,3,1
3,4,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping,37,5,9,1
4,5,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference,37,5,45,1


*appleStore.csv*


"id": ID do aplicativo

"track_name": nome do aplicativo

"size_bytes": tamanho (em bytes)

"currency": tipo de moeda

"preço": valor do preço

"ratingcounttot": contagens de classificação do usuário (para todas as versões)

"ratingcountver": contagens de classificação do usuário (para versão atual)

"user_rating": valor médio da classificação do usuário (para todas as versões)

"userratingver": valor médio da classificação do usuário (para versão atual)

"ver": código da versão mais recente

"cont_rating": classificação do conteúdo

"prime_genre": gênero principal

"sup_devices.num": Número de dispositivos de suporte

"ipadSc_urls.num": número de capturas de tela exibidas para exibição

"lang.num": número de idiomas suportados

"vpp_lic": licenciamento baseado em dispositivo Vpp ativado


*appleStore_description.csv*

id : App ID

track_name: Application name

size_bytes: Memory size (in Bytes)

app_desc: Application description

# Filtrando dados

In [3]:
apps = apps.drop(columns = ['sup_devices.num','ipadSc_urls.num','lang.num','vpp_lic','Unnamed: 0'])
apps.head()

,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre
0,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games
1,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping
4,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference


In [4]:
apps.columns = ['ID', 'nome','tamanho_bytes','moeda','preco',
                 'avaliacoes','avaliacoes_ultima_versao','nota',
                 'nota_versao','versao','classificacao','genero']
apps.head()
# Rever depois a coluna nota e media_avaliacoes tem algo de errado

,ID,nome,tamanho_bytes,moeda,preco,avaliacoes,avaliacoes_ultima_versao,nota,nota_versao,versao,classificacao,genero
0,281656475,PAC-MAN Premium,100788224,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games
1,281796108,Evernote - stay organized,158578688,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping
4,282935706,Bible,92774400,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference


In [5]:
apps_descricao.head()

,id,track_name,size_bytes,app_desc
0,281656475,PAC-MAN Premium,100788224,"SAVE 20%, now only $3.99 for a limited time!\n..."
1,281796108,Evernote - stay organized,158578688,Let Evernote change the way you organize your ...
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,Download the most popular free weather app pow...
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,The eBay app is the best way to find anything ...
4,282935706,Bible,92774400,On more than 250 million devices around the wo...


In [6]:
apps_descricao.columns = ['ID','nome','tamanho_bytes', 'descricao']
# Como tamanho_bytes e nome já existe em app, vou dropar no app, e deixar a informação apenas nesse. o ID necessita ficar em ambos
apps = apps.drop(columns = ['nome','tamanho_bytes'])
apps.head()

,ID,moeda,preco,avaliacoes,avaliacoes_ultima_versao,nota,nota_versao,versao,classificacao,genero
0,281656475,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games
1,281796108,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity
2,281940292,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather
3,282614216,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping
4,282935706,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference


In [7]:
apps_descricao.head()

,ID,nome,tamanho_bytes,descricao
0,281656475,PAC-MAN Premium,100788224,"SAVE 20%, now only $3.99 for a limited time!\n..."
1,281796108,Evernote - stay organized,158578688,Let Evernote change the way you organize your ...
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",100524032,Download the most popular free weather app pow...
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",128512000,The eBay app is the best way to find anything ...
4,282935706,Bible,92774400,On more than 250 million devices around the wo...


Transforma bytes para megabytes(kb para mb)

In [8]:
# 1 megabytes = 1kb * 1e-6
apps_descricao['tamanho_megabytes'] = apps_descricao['tamanho_bytes'] * 1e-6
apps_descricao = apps_descricao.drop(columns = ['tamanho_bytes'])
apps_descricao.head()

,ID,nome,descricao,tamanho_megabytes
0,281656475,PAC-MAN Premium,"SAVE 20%, now only $3.99 for a limited time!\n...",100.788224
1,281796108,Evernote - stay organized,Let Evernote change the way you organize your ...,158.578688
2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",Download the most popular free weather app pow...,100.524032
3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",The eBay app is the best way to find anything ...,128.512000
4,282935706,Bible,On more than 250 million devices around the wo...,92.774400


In [9]:
apps.head()

,ID,moeda,preco,avaliacoes,avaliacoes_ultima_versao,nota,nota_versao,versao,classificacao,genero
0,281656475,USD,3.99,21292,26,4.0,4.5,6.3.5,4+,Games
1,281796108,USD,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity
2,281940292,USD,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather
3,282614216,USD,0.00,262241,649,4.0,4.5,5.10.0,12+,Shopping
4,282935706,USD,0.00,985920,5320,4.5,5.0,7.5.1,4+,Reference


In [10]:
gratuito = apps['preco'] == 0.0
apps['gratuito'] = gratuito
apps.tail()

,ID,moeda,preco,avaliacoes,avaliacoes_ultima_versao,nota,nota_versao,versao,classificacao,genero,gratuito
7192,1187617475,USD,0.00,142,75,4.5,4.5,1.3,4+,Games,True
7193,1187682390,USD,0.00,30,30,4.5,4.5,0.9,4+,Games,True
7194,1187779532,USD,1.99,15,0,4.5,0.0,1.0.2,9+,Utilities,False
7195,1187838770,USD,0.00,85,32,4.5,4.5,1.0.15,12+,Games,True
7196,1188375727,USD,0.00,3,3,5.0,5.0,1.0,4+,Games,True


In [28]:
apps_descricao = apps_descricao.drop(columns = ['ID'],errors='ignore')
apps = apps.drop(columns = ['ID','moeda'],errors='ignore')

apps_descricao['preco'] = apps['preco']
apps_descricao['nota'] = apps['nota']

apps_descricao.head(10)

,nome,descricao,tamanho_megabytes,preco,nota
0,PAC-MAN Premium,"SAVE 20%, now only $3.99 for a limited time!\n...",100.788224,3.99,4.0
1,Evernote - stay organized,Let Evernote change the way you organize your ...,158.578688,0.00,4.0
2,"WeatherBug - Local Weather, Radar, Maps, Alerts",Download the most popular free weather app pow...,100.524032,0.00,3.5
3,"eBay: Best App to Buy, Sell, Save! Online Shop...",The eBay app is the best way to find anything ...,128.512000,0.00,4.0
4,Bible,On more than 250 million devices around the wo...,92.774400,0.00,4.5
5,Shanghai Mahjong,"★ WINNER ""BEST GAME"" 2009\n★ 3rd PLACE WINNER ...",10.485713,0.99,4.0
6,PayPal - Send and request money safely,Description\nTAP INTO YOUR MONEY\nSend money o...,227.795968,0.00,4.0
7,Pandora - Music & Radio,Find the music you love and let the music you ...,130.242560,0.00,4.0
8,PCalc - The Best Calculator,"PCalc is the powerful choice for scientists, e...",49.250304,9.99,4.5
9,Ms. PAC-MAN,Now with MFi controller support!\n\nMs. PAC-MA...,70.023168,3.99,4.0


# Classificações:.

* Genero
* Rentaveis
* Top gratuitos
* Melhores notas
* Melhores preços
* Tamanho do arquivo

# Genero
* Filtrar pelo genero 
* Se o usuario quer gratuito ou não
* Classificar em ordem pela nota

In [29]:
generos = apps.genero.unique().tolist()
generos

['Games',
 'Productivity',
 'Weather',
 'Shopping',
 'Reference',
 'Finance',
 'Music',
 'Utilities',
 'Travel',
 'Social Networking',
 'Sports',
 'Business',
 'Health & Fitness',
 'Entertainment',
 'Photo & Video',
 'Navigation',
 'Education',
 'Lifestyle',
 'Food & Drink',
 'News',
 'Book',
 'Medical',
 'Catalogs']

In [30]:
apps.describe()

,preco,avaliacoes,avaliacoes_ultima_versao,nota,nota_versao
count,7197.000000,7.197000e+03,7197.000000,7197.000000,7197.000000
mean,1.726218,1.289291e+04,460.373906,3.526956,3.253578
std,5.833006,7.573941e+04,3920.455183,1.517948,1.809363
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,2.800000e+01,1.000000,3.500000,2.500000
50%,0.000000,3.000000e+02,23.000000,4.000000,4.000000
75%,1.990000,2.793000e+03,140.000000,4.500000,4.500000
max,299.990000,2.974676e+06,177050.000000,5.000000,5.000000


In [0]:
def achar_valores(df):
  index = df.index.tolist()
  return apps_descricao.iloc[index]

In [0]:
def recomendacao_por_genero(genero_escolhido,app_gratuito = None,apps = apps, k = 10):
  apps_famosos = apps.query('avaliacoes >= avaliacoes.mean()')

  if (app_gratuito):
    apps_famosos = apps_famosos.query('gratuito == True')

  apps_famosos = apps_famosos.query(f'genero == "{genero_escolhido}"')
  apps_recomendados = apps_famosos.sort_values('nota', ascending = False).head(k)

  return achar_valores(apps_recomendados)

In [52]:
# Pedi o genero dos jogos, indiferente se é pago ou gratuito e  quero uma sample de 15 amostras
recomendacao_por_genero(generos[0], k = 15)

,nome,descricao,tamanho_megabytes,preco,nota
1827,X-War: Clash of Zombies,Clash of Zombies is an exciting SLG with Super...,42.917888,0.00,5.0
5690,PewDiePie's Tuber Simulator,Follow in the footsteps of one of the biggest ...,266.766336,0.00,5.0
3086,Sniper 3D Assassin: Shoot to Kill Gun Game,Ready to have FUN? Download now the best shoot...,157.851648,0.00,5.0
350,Plants vs. Zombies HD,The game requires iOS 6 compatible device.\n\n...,225.859584,0.99,5.0
5001,Yu-Gi-Oh! Duel Links,Take on global Duelists in real-time and while...,185.152512,0.00,5.0
4457,Vlogger Go Viral - Clicker Game & Vlog Simulator,Get ready to become the next Internet sensatio...,100.562944,0.00,5.0
2064,The Room Two,"WARNING: Due to heavy graphics requirements, T...",460.077056,1.99,5.0
4990,Gin Rummy Plus - Multiplayer Online Card Game,Join the world’s most popular Gin Rummy Game a...,178.644992,0.00,5.0
1484,The Room,"Welcome to The Room, a physical puzzler, wrapp...",338.273280,0.99,5.0
2989,The Room Three,***WARNING: DUE TO HEAVY GRAPHICS REQUIREMENTS...,1005.560832,4.99,5.0


In [53]:
recomendacao_por_genero(generos[2])

,nome,descricao,tamanho_megabytes,preco,nota
930,"NOAA Hi-Def Radar Pro - Storm Warnings, Hurri...",Limited Time Sale - 50% OFF - DOWNLOAD NOW!\nS...,27.500544,1.99,5.0
172,MyRadar NOAA Weather Radar Forecast,Thanks to everyone for making MyRadar so succe...,127.284224,0.00,4.5
182,"MyRadar Pro NOAA Weather Radar, Forecasts & St...",Thanks to everyone for making MyRadar so succe...,123.718656,1.99,4.5
1095,Weather Underground: Custom Forecast & Local R...,"Weather Underground for iPhone, iPad, and Appl...",99.829760,0.00,4.5
1890,Yahoo Weather,"Experience the weather | Apple Design Award, 2...",123.047936,0.00,4.5
2309,"Weather Live - Weather Forecast, Radar and Alerts",Meet Weather Live. The most beautiful weather ...,57.539584,4.99,4.5
2310,Weather Live Free - Weather Forecast & Alerts,Meet the free version of Weather Live. The mos...,101.524480,0.00,4.5
2319,NOAA Radar Pro – Weather Alerts & Forecast,LIMITED TIME SALE! - 30% OFF! - DOWNLOAD NOW! ...,82.765824,3.99,4.5
2320,NOAA Weather Radar - Weather Forecast & HD Radar,NOAA Weather Radar is a powerful yet easy-to-u...,108.172288,0.00,4.5
3355,Storm Radar,Keep your eye on the sky with Storm Radar by W...,93.304832,0.00,4.5


In [58]:
recomendacao_por_genero(generos[2],app_gratuito = True, k = 5)

,nome,descricao,tamanho_megabytes,preco,nota
172,MyRadar NOAA Weather Radar Forecast,Thanks to everyone for making MyRadar so succe...,127.284224,0.0,4.5
1095,Weather Underground: Custom Forecast & Local R...,"Weather Underground for iPhone, iPad, and Appl...",99.829760,0.0,4.5
1890,Yahoo Weather,"Experience the weather | Apple Design Award, 2...",123.047936,0.0,4.5
2310,Weather Live Free - Weather Forecast & Alerts,Meet the free version of Weather Live. The mos...,101.524480,0.0,4.5
2320,NOAA Weather Radar - Weather Forecast & HD Radar,NOAA Weather Radar is a powerful yet easy-to-u...,108.172288,0.0,4.5


# Rentaveis

* Mais votos
* Ter um grande numero de avaliações
* Ter um grande numero de avaliações na ultima versão
* Nota da ultima versão tem que ser boa
* Ordenar por nota total


Celulas anteriores foram rodadas novamente por questão de organização

In [63]:
apps.head(3)

,preco,avaliacoes,avaliacoes_ultima_versao,nota,nota_versao,versao,classificacao,genero,gratuito
0,3.99,21292,26,4.0,4.5,6.3.5,4+,Games,False
1,0.00,161065,26,4.0,3.5,8.2.2,4+,Productivity,True
2,0.00,188583,2822,3.5,4.5,5.0.0,4+,Weather,True


In [64]:
apps_descricao.head(3)

,nome,descricao,tamanho_megabytes,preco,nota
0,PAC-MAN Premium,"SAVE 20%, now only $3.99 for a limited time!\n...",100.788224,3.99,4.0
1,Evernote - stay organized,Let Evernote change the way you organize your ...,158.578688,0.00,4.0
2,"WeatherBug - Local Weather, Radar, Maps, Alerts",Download the most popular free weather app pow...,100.524032,0.00,3.5


In [65]:
apps.describe()

,preco,avaliacoes,avaliacoes_ultima_versao,nota,nota_versao
count,7197.000000,7.197000e+03,7197.000000,7197.000000,7197.000000
mean,1.726218,1.289291e+04,460.373906,3.526956,3.253578
std,5.833006,7.573941e+04,3920.455183,1.517948,1.809363
min,0.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000,2.800000e+01,1.000000,3.500000,2.500000
50%,0.000000,3.000000e+02,23.000000,4.000000,4.000000
75%,1.990000,2.793000e+03,140.000000,4.500000,4.500000
max,299.990000,2.974676e+06,177050.000000,5.000000,5.000000


In [59]:
generos = apps.genero.unique().tolist()
generos

['Games',
 'Productivity',
 'Weather',
 'Shopping',
 'Reference',
 'Finance',
 'Music',
 'Utilities',
 'Travel',
 'Social Networking',
 'Sports',
 'Business',
 'Health & Fitness',
 'Entertainment',
 'Photo & Video',
 'Navigation',
 'Education',
 'Lifestyle',
 'Food & Drink',
 'News',
 'Book',
 'Medical',
 'Catalogs']

In [0]:
def achar_valores(df):
  index = df.index.tolist()
  return apps_descricao.iloc[index]

In [0]:
def recomendacao_rentaveis(genero_escolhido = None,app_gratuito = None,apps = apps, k = 10):
  apps_famosos = apps.query('avaliacoes >= avaliacoes.mean()')
  apps_famosos = apps.query('avaliacoes_ultima_versao >= avaliacoes_ultima_versao.std() and nota_versao >= nota_versao.std()')


  if (app_gratuito):
    apps_famosos = apps_famosos.query('gratuito == True')

  if (genero_escolhido):
      apps_famosos = apps_famosos.query(f'genero == "{genero_escolhido}"')



  apps_recomendados = apps_famosos.sort_values('nota', ascending = False).head(k)
  return achar_valores(apps_recomendados)
  #return apps_recomendados

In [85]:
recomendacao_rentaveis(genero_escolhido = 0, k = 20)

,nome,descricao,tamanho_megabytes,preco,nota
1877,Geometry Dash,Jump and fly your way through danger in this r...,83.931136,1.99,5.0
271,TurboScan™ Pro - document & receipt scanner: s...,TurboScan turns your iPhone into a full-featur...,8.821760,4.99,5.0
3389,Flight Pilot Simulator 3D: Flying Game For Free,"Unleash your inner Maverick, become a pilot an...",142.512128,0.00,5.0
3616,Browser and File Manager for Documents,"Download and browse files on your iPhone, iPod...",42.198016,0.00,5.0
3784,"Egg, Inc.","In the near future, the secrets of the univers...",53.579776,0.00,5.0
4263,Super Sharp,50% off for a limited time as part of the App ...,15.110144,1.99,5.0
4393,Fitbod Workout Log: Strength Lifting Bodybuilding,Featured by Apple in “New Apps We Love”. Find ...,71.352320,0.00,5.0
545,"Zappos: shop shoes & clothes, fast free shipping",Our shopping app does its darndest to deliver ...,70.325248,0.00,5.0
498,Infinity Blade,Winner of the Apple Design Award and 30+ “Game...,624.107810,0.99,5.0
3086,Sniper 3D Assassin: Shoot to Kill Gun Game,Ready to have FUN? Download now the best shoot...,157.851648,0.00,5.0


In [86]:
recomendacao_rentaveis(app_gratuito = True, k = 20)

,nome,descricao,tamanho_megabytes,preco,nota
3616,Browser and File Manager for Documents,"Download and browse files on your iPhone, iPod...",42.198016,0.0,5.0
3086,Sniper 3D Assassin: Shoot to Kill Gun Game,Ready to have FUN? Download now the best shoot...,157.851648,0.0,5.0
3389,Flight Pilot Simulator 3D: Flying Game For Free,"Unleash your inner Maverick, become a pilot an...",142.512128,0.0,5.0
6688,Tricky Test 2™: Genius Brain?,Apple Featured Game as New Games We Love in iT...,134.669312,0.0,5.0
3784,"Egg, Inc.","In the near future, the secrets of the univers...",53.579776,0.0,5.0
545,"Zappos: shop shoes & clothes, fast free shipping",Our shopping app does its darndest to deliver ...,70.325248,0.0,5.0
4393,Fitbod Workout Log: Strength Lifting Bodybuilding,Featured by Apple in “New Apps We Love”. Find ...,71.352320,0.0,5.0
1104,Head Soccer,◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆\nApp store(...,121.319424,0.0,5.0
6663,Super Cat Tales,Six cats are the new Jump'n'Fur stars!\n \nReu...,64.337920,0.0,5.0
4969,Good Knight Story,What happened last night!? Join our unlikely h...,153.513984,0.0,5.0


In [91]:
recomendacao_rentaveis(genero_escolhido = 'Finance', k = 20)
# Apenas 1 app atende os requisitos

,nome,descricao,tamanho_megabytes,preco,nota
67,Chase Mobile℠,Do your banking right from your mobile device\...,39.50592,0.0,4.5


# Melhores preços

# Filtrar pelo tamanho do arquivo